## Import packages

In [193]:
import selenium
from selenium import webdriver

import shutil, time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [172]:
path_to_extension = r'C:\Users\Matth\Desktop\3.12_0'
character_name = "Mercychan"
character_server = "Benediction"
character_region = "US"

clear_char_data = True

## Navigate to character page

In [189]:
char_url, browser = src.load_environment(path_to_extension, character_name, character_server, character_region)

In [123]:
bosses = src.get_phase3_bosses(browser)

In [230]:
# Feel free to omit bosses that you are not interested in
bosses = ["Shade of Akama"]#, "Anetheron", "The Illidari Council"]

In [227]:
boss_link_dict = {"High Warlord Najentus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=610", "Anetheron" : "#boss=611", "Kaz'rogal" : "#boss=612", 
                  "Azgalor" : "#boss=613", "Archimonde" : "#boss=614"}

In [124]:
char_info = src.get_character_info(char_url)
char_info

'Mercychan (Benediction US)'

## Main code loop

In [231]:
def main():
    if clear_char_data:
        src.clear_character_data()
    
    for i, boss in enumerate(bosses):
        
        player_df = pd.DataFrame(pd.np.empty((0, 15)))
        player_df.columns = ["Name", "Server", "Kill time", "Rank", "nHealers", "Spriest?", "Innervate?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotations"]

        browser.get(f"https://classic.warcraftlogs.com/character/{character_region.lower()}/{character_server.lower()}/{character_name.lower()}{boss_link_dict[boss]}")
        time.sleep(1)

        # Pick out the top parse of this boss
        search = browser.find_elements_by_class_name("character-table-link")
        boss_page_url = browser.current_url
        
        #for i in range(len(search)):
        for i in range(1):
                        
            try:
                rank, boss_rank, boss_HPS, boss_killtime = src.get_boss_data(browser, i)

            except IndexError: continue
            
            rank.click()
            time.sleep(2)
            
            temp_url = browser.current_url
            
            player_HPS = src.get_player_HPS(browser, character_name)
            
            boss_tanks = get_tanks(browser)
            nHealers = get_nHealers(browser)
            
            print(f"Boss: {boss}")
            print(f"Tanks: {boss_tanks}")

            '''
            all_tanks = src.get_tanks(browser) # Returns the 3 tank names for the current raid

            if boss in ["Anetheron", "Azgalor", "Reliquary of Souls"]:
                boss_tanks = src.get_boss_tanks(browser, all_tanks, 2) 

            elif boss in ["Shade of Akama", "The Illidari Council", "Illidan Stormrage"]:
                boss_tanks = src.get_boss_tanks(browser, all_tanks, 3) 

            else:
                boss_tanks = src.get_boss_tanks(browser, all_tanks, 1) 
            '''

            browser.get(temp_url)
            time.sleep(0.5)

            src.scroll_click(browser.find_element_by_link_text(character_name))
            time.sleep(0.5)
            
            #player_HPS = src.get_player_HPS(browser, character_name)
            #time.sleep(1)
            
            LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, player_HPS)
            time.sleep(2)
                
            spriest = src.check_spriest(browser)
            innervate = src.check_innervate(browser)

            # Download the cast-sequence and damage-taken CSVs.
            src.download_csv(browser, temp_url, "filter-casts-tab", "cast_sequence/casts.csv")
            src.download_csv(browser, temp_url, "filter-damage-taken-tab", "damage_taken/dmg_taken.csv")

            #nHealers = src.get_nHealers(browser)
            browser.get(temp_url)
            time.sleep(1)

            # For most single tank bosses, the main tank is the current boss' melee target
            if boss in ["Rage Winterchill", "Kaz'rogal", "Archimonde", "High Warlord Naj'entus", "Supremus", 
                        "Teron Gorefiend", "Gurtogg Bloodboil", "Mother Shahraz"]:

                # Clean the csvs
                df = src.clean_dmg_taken_csv(boss)
                df2 = src.clean_cast_sequence_csv()

                # Fix regrowth timer
                df2 = src.fix_regrowth_cast_time(df2)

                # Add boss target
                df = src.add_boss_target(boss, df, df2)
                df = src.fix_boss_target(df)

                # Get the rotations
                try: rotations = src.calculate_rotations(df, boss, boss_tanks)
                except ZeroDivisionError:
                    print("Click element error. Skipping this parse.")
                    browser.get(boss_page_url)
                    time.sleep(0.5)
                    continue

            elif boss in ["Anetheron", "Azgalor", "Reliquary of Souls",
                         "Shade of Akama", "The Illidari Council", "Illidan Stormrage"]:

                # Clean the csv
                df = src.clean_cast_sequence_csv()
                df = src.fix_regrowth_cast_time(df)

                # Get the rotations
                try: rotations = src.calculate_rotations(df, boss, boss_tanks)
                except ZeroDivisionError:
                    print("Click element error. Skipping this parse.")
                    browser.get(boss_page_url)
                    time.sleep(0.5)
                    continue

            print(f'Rotations: {rotations}')    

            # Save data to csv
            to_append = [character_name, character_server + " " + character_region, boss_killtime, boss_rank, str(nHealers), spriest, innervate, LB_uptime, boss_HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotations]
            series = pd.Series(to_append, index = player_df.columns)
            player_df = player_df.append(series, ignore_index=True)

            #Return to boss page
            browser.get(boss_page_url)
            time.sleep(0.5)
            print('------')
                
        # Return to character page
        browser.get(char_url)
        time.sleep(0.5)

        player_df.to_csv(f"character_data/{boss}_{character_name}.csv", index=None)
        
    src.combine_character_data(character_name)
    

if __name__ == "__main__":
    main()

Boss: Shade of Akama
Tanks: ['Armadomin', 'Hilfedanke', 'Kryd']
Rotations: [['Other', 0.481], ['1LB 2RG', 0.074]]
------


In [218]:
def get_tanks(browser):
    src.click_on_element_by_id(browser, "filter-summary-tab")  
    time.sleep(0.5)
    
    row = browser.find_elements_by_class_name("composition-row")
    
    tanks = row[0].text.split("\n")
    tanks.pop(0)
    
    return tanks

def get_nHealers(browser):
    row = browser.find_elements_by_class_name("composition-row")
    
    return len(row[2].text.split("\n")) - 1